In [1]:
import sys.process._
// Работает с датасетом, созданным в Лабе 4
println("hdfs dfs -ls /labs/laba03/".!!)

Found 2 items
-rw-r--r--   3 hdfs hdfs  563826904 2020-05-14 18:12 /labs/laba03/weblogs.json
drwxr-xr-x   - hdfs hdfs          0 2020-05-15 01:07 /labs/laba03/weblogs.parquet



In [8]:
val logs = spark.read.parquet("hdfs:///labs/laba03/weblogs.parquet")

logs = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]


[uid: string, visits: array<struct<timestamp:bigint,url:string>>]

In [11]:
logs.show(1,200, false)

+------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                 uid|                                                                                                                                                                                                  visits|
+------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|39d8aded-2111-42ab-9393-37c4f781d030|[[1419590902274, http://go.mail.ru/], [1419590344372, http://thephilanews.com/ru/wawa-welcome-america-begins-with-reading-…-and-the-runway-32987.htm], [1419590203997, http://visitorsworth.com/www.w...|
+------------------------------------+--

In [9]:
logs.printSchema

root
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- timestamp: long (nullable = true)
 |    |    |-- url: string (nullable = true)



In [10]:
import org.apache.spark.sql.functions._

val logs_2 = logs.select($"uid", explode($"visits")).select("uid", "col.timestamp", "col.url")

logs_2 = [uid: string, timestamp: bigint ... 1 more field]


[uid: string, timestamp: bigint ... 1 more field]

In [22]:
logs_2.filter('uid ==="39d8aded-2111-42ab-9393-37c4f781d030").show(20,200,false)

+------------------------------------+-------------+----------------------------------------------------------------------------------------------+
|                                 uid|    timestamp|                                                                                           url|
+------------------------------------+-------------+----------------------------------------------------------------------------------------------+
|39d8aded-2111-42ab-9393-37c4f781d030|1419590902274|                                                                            http://go.mail.ru/|
|39d8aded-2111-42ab-9393-37c4f781d030|1419590344372|http://thephilanews.com/ru/wawa-welcome-america-begins-with-reading-…-and-the-runway-32987.htm|
|39d8aded-2111-42ab-9393-37c4f781d030|1419590203997|                                                         http://visitorsworth.com/www.wawa.com|
|39d8aded-2111-42ab-9393-37c4f781d030|1419589141914|                             http://10receptov.net/rojdestve

In [11]:
// парсим сайт
val log_3 = logs_2
        .withColumn("host", lower(callUDF("parse_url", $"url", lit("HOST"))))
        .withColumn("domain", regexp_replace($"host", "www.", ""))

log_3 = [uid: string, timestamp: bigint ... 3 more fields]


[uid: string, timestamp: bigint ... 3 more fields]

In [12]:
log_3.filter('uid ==="39d8aded-2111-42ab-9393-37c4f781d030").show(20)

+--------------------+-------------+--------------------+-----------------+-----------------+
|                 uid|    timestamp|                 url|             host|           domain|
+--------------------+-------------+--------------------+-----------------+-----------------+
|39d8aded-2111-42a...|1419590902274|  http://go.mail.ru/|       go.mail.ru|       go.mail.ru|
|39d8aded-2111-42a...|1419590344372|http://thephilane...| thephilanews.com| thephilanews.com|
|39d8aded-2111-42a...|1419590203997|http://visitorswo...|visitorsworth.com|visitorsworth.com|
|39d8aded-2111-42a...|1419589141914|http://10receptov...|   10receptov.net|   10receptov.net|
|39d8aded-2111-42a...|1419588542058|http://www.koolin...|  www.koolinar.ru|      koolinar.ru|
|39d8aded-2111-42a...|1427181952000|http://www.mnogo....|     www.mnogo.ru|         mnogo.ru|
|39d8aded-2111-42a...|1427181928000|http://www.mnogo....|     www.mnogo.ru|         mnogo.ru|
|39d8aded-2111-42a...|1427181927000|http://www.mnogo....|   

#### ТОП-1000 по сайтам

In [13]:
val toplog = log_3.groupBy('domain).count() 

toplog = [domain: string, count: bigint]


[domain: string, count: bigint]

In [14]:
toplog.sort($"count".desc)show(30)

+--------------------+------+
|              domain| count|
+--------------------+------+
|            avito.ru|352568|
|                http|349733|
|          smotri.com|186882|
|                null|128051|
|               https| 86614|
|           24open.ru| 76716|
|       loveplanet.ru| 74833|
|     mail.rambler.ru| 73418|
|              vk.com| 70608|
|         youtube.com| 63539|
|           yandex.ru| 56345|
|         echo.msk.ru| 40039|
|        yaplakal.com| 37226|
|           yabadu.ru| 34483|
|            ebay.com| 34434|
|         svyaznoy.ru| 32360|
|  forum.littleone.ru| 32290|
|        seasonvar.ru| 31367|
|             ozon.ru| 30601|
|               vz.ru| 29997|
|         topface.com| 28270|
|        b.kavanga.ru| 26413|
|             adme.ru| 25910|
|   go.youlamedia.com| 25375|
|         mail.qip.ru| 25233|
|           sports.ru| 23104|
|content.adspynet.com| 22875|
|    news.sportbox.ru| 22850|
|              999.md| 21565|
|         xvideos.com| 20696|
+---------

In [30]:
toplog.count

106333

In [33]:
toplog.filter('domain.isNull).show

+------+------+
|domain| count|
+------+------+
|  null|128051|
+------+------+



In [15]:
val toplog2 = toplog.filter('domain.isNotNull).sort($"count".desc).limit(1000)

toplog2 = [domain: string, count: bigint]


[domain: string, count: bigint]

In [16]:
toplog2.count

1000

In [41]:
toplog2.show

+------------------+------+
|            domain| count|
+------------------+------+
|          avito.ru|352568|
|              http|349733|
|        smotri.com|186882|
|             https| 86614|
|         24open.ru| 76716|
|     loveplanet.ru| 74833|
|   mail.rambler.ru| 73418|
|            vk.com| 70608|
|       youtube.com| 63539|
|         yandex.ru| 56345|
|       echo.msk.ru| 40039|
|      yaplakal.com| 37226|
|         yabadu.ru| 34483|
|          ebay.com| 34434|
|       svyaznoy.ru| 32360|
|forum.littleone.ru| 32290|
|      seasonvar.ru| 31367|
|           ozon.ru| 30601|
|             vz.ru| 29997|
|       topface.com| 28270|
+------------------+------+
only showing top 20 rows



##### как собрать в один вектор столбец

In [18]:
toplog2
  //.groupBy()
  .agg(collect_list("domain") as "domain1")
  .show(1, 100, false)

+----------------------------------------------------------------------------------------------------+
|                                                                                             domain1|
+----------------------------------------------------------------------------------------------------+
|[avito.ru, http, smotri.com, https, 24open.ru, loveplanet.ru, mail.rambler.ru, vk.com, youtube.co...|
+----------------------------------------------------------------------------------------------------+



#### Список клиентов

In [19]:
// уник юзеров
logs.select(countDistinct("uid")).show

+-------------------+
|count(DISTINCT uid)|
+-------------------+
|              36138|
+-------------------+



In [20]:
val clients = logs.select('uid)
clients.count

clients = [uid: string]


36138

#### Джойним клиентов на сайты из ТОП-1000

In [21]:
val cl_top = clients.crossJoin(toplog2)

cl_top = [uid: string, domain: string ... 1 more field]


[uid: string, domain: string ... 1 more field]

In [58]:
cl_top.filter('uid ==="39d8aded-2111-42ab-9393-37c4f781d030").show(20)

+--------------------+------------------+------+
|                 uid|            domain| count|
+--------------------+------------------+------+
|39d8aded-2111-42a...|          avito.ru|352568|
|39d8aded-2111-42a...|              http|349733|
|39d8aded-2111-42a...|        smotri.com|186882|
|39d8aded-2111-42a...|             https| 86614|
|39d8aded-2111-42a...|         24open.ru| 76716|
|39d8aded-2111-42a...|     loveplanet.ru| 74833|
|39d8aded-2111-42a...|   mail.rambler.ru| 73418|
|39d8aded-2111-42a...|            vk.com| 70608|
|39d8aded-2111-42a...|       youtube.com| 63539|
|39d8aded-2111-42a...|         yandex.ru| 56345|
|39d8aded-2111-42a...|       echo.msk.ru| 40039|
|39d8aded-2111-42a...|      yaplakal.com| 37226|
|39d8aded-2111-42a...|         yabadu.ru| 34483|
|39d8aded-2111-42a...|          ebay.com| 34434|
|39d8aded-2111-42a...|       svyaznoy.ru| 32360|
|39d8aded-2111-42a...|forum.littleone.ru| 32290|
|39d8aded-2111-42a...|      seasonvar.ru| 31367|
|39d8aded-2111-42a..

#### Посчитаем кол-во посещений

In [22]:
import org.apache.spark.sql.expressions.Window

// для проверки порядка сделаем row_number
val windowSpec = Window.partitionBy("uid").orderBy("domain")

windowSpec = org.apache.spark.sql.expressions.WindowSpec@19f79d23


org.apache.spark.sql.expressions.WindowSpec@19f79d23

In [24]:
val cl_topjoin = cl_top.as('a)
        .join(log_3.as('b), $"a.uid" === $"b.uid" && $"a.domain" === $"b.domain", "left")
        .groupBy($"a.uid", $"a.domain")
        .agg(count($"b.uid").as("cnt"))
        .na.fill(0)
        .sort($"uid", $"domain")
        .withColumn("row_number", row_number().over(windowSpec))

cl_topjoin = [uid: string, domain: string ... 2 more fields]


[uid: string, domain: string ... 2 more fields]

In [25]:
cl_topjoin.printSchema

root
 |-- uid: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- cnt: long (nullable = false)
 |-- row_number: integer (nullable = true)



##### проверка

In [26]:
cl_topjoin.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030" && 'cnt >0).show(false)

+------------------------------------+-----------+---+----------+
|uid                                 |domain     |cnt|row_number|
+------------------------------------+-----------+---+----------+
|39d8aded-2111-42ab-9393-37c4f781d030|go.mail.ru |1  |312       |
|39d8aded-2111-42ab-9393-37c4f781d030|koolinar.ru|1  |423       |
|39d8aded-2111-42ab-9393-37c4f781d030|mnogo.ru   |3  |536       |
+------------------------------------+-----------+---+----------+



In [27]:
cl_topjoin.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").count

1000

In [28]:
cl_topjoin.select(countDistinct("uid"), count("uid")).show

+-------------------+----------+
|count(DISTINCT uid)|count(uid)|
+-------------------+----------+
|              36138|  36138000|
+-------------------+----------+



In [106]:
cl_topjoin.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").show()

+--------------------+----------------+---+----------+
|                 uid|          domain|cnt|row_number|
+--------------------+----------------+---+----------+
|39d8aded-2111-42a...|    .kasparov.ru|  0|         1|
|39d8aded-2111-42a...|     0629.com.ua|  0|         2|
|39d8aded-2111-42a...|     1001eda.com|  0|         3|
|39d8aded-2111-42a...|    1001golos.ru|  0|         4|
|39d8aded-2111-42a...| 1001goroskop.ru|  0|         5|
|39d8aded-2111-42a...|      1001tur.ru|  0|         6|
|39d8aded-2111-42a...|          101.ru|  0|         7|
|39d8aded-2111-42a...|        11x11.ru|  0|         8|
|39d8aded-2111-42a...|123greetings.com|  0|         9|
|39d8aded-2111-42a...|   123magazin.ru|  0|        10|
|39d8aded-2111-42a...|       1prime.ru|  0|        11|
|39d8aded-2111-42a...|      2012god.ru|  0|        12|
|39d8aded-2111-42a...| 2015godkozy.com|  0|        13|
|39d8aded-2111-42a...|         24au.ru|  0|        14|
|39d8aded-2111-42a...|       24open.ru|  0|        15|
|39d8aded-

#### Собираем в один вектор cnt по отсортированной колонке domain

In [29]:
val cl_topjoin2 = cl_topjoin
  .groupBy("uid")
  .agg(collect_list("cnt") as "domain_features")

cl_topjoin2 = [uid: string, domain_features: array<bigint>]


[uid: string, domain_features: array<bigint>]

In [95]:
cl_topjoin2.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").show(100)

+--------------------+--------------------+
|                 uid|     domain_features|
+--------------------+--------------------+
|39d8aded-2111-42a...|[0, 0, 0, 0, 0, 0...|
+--------------------+--------------------+



In [30]:
cl_topjoin2.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030")
  .select(
    $"domain_features".getItem(312-1).as("col1"),
    $"domain_features".getItem(423-1).as("col2"),
    $"domain_features".getItem(536-1).as("col3")
      )
.show

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   1|   3|
+----+----+----+



#### domain_features Ок

#### Достаем данные из лабы 05
Слейте этот датасет с матрицей users-items из лабы 05 (версия 20200429, которая должна храниться у вас в домашней директории HDFS в подпапке visits) по ключу uid, чтобы получился один большой датасет.

In [3]:
import sys.process._

println("hdfs dfs -ls /user/vladimir.dyukarev/users-items/".!!)

Found 1 items
drwxr-xr-x   - vladimir.dyukarev vladimir.dyukarev          0 2020-06-27 19:38 /user/vladimir.dyukarev/users-items/20200429



In [5]:
val visits = spark.read.parquet("hdfs:////user/vladimir.dyukarev/users-items/20200429")

Waiting for a Spark session to start...

visits = [uid: string, buy_cameras_0: bigint ... 639 more fields]


[uid: string, buy_cameras_0: bigint ... 639 more fields]

In [6]:
visits.printSchema

root
 |-- uid: string (nullable = true)
 |-- buy_cameras_0: long (nullable = true)
 |-- buy_cameras_1: long (nullable = true)
 |-- buy_cameras_10: long (nullable = true)
 |-- buy_cameras_11: long (nullable = true)
 |-- buy_cameras_12: long (nullable = true)
 |-- buy_cameras_13: long (nullable = true)
 |-- buy_cameras_14: long (nullable = true)
 |-- buy_cameras_15: long (nullable = true)
 |-- buy_cameras_16: long (nullable = true)
 |-- buy_cameras_17: long (nullable = true)
 |-- buy_cameras_18: long (nullable = true)
 |-- buy_cameras_19: long (nullable = true)
 |-- buy_cameras_2: long (nullable = true)
 |-- buy_cameras_3: long (nullable = true)
 |-- buy_cameras_4: long (nullable = true)
 |-- buy_cameras_5: long (nullable = true)
 |-- buy_cameras_6: long (nullable = true)
 |-- buy_cameras_7: long (nullable = true)
 |-- buy_cameras_8: long (nullable = true)
 |-- buy_cameras_9: long (nullable = true)
 |-- buy_clothing_0: long (nullable = true)
 |-- buy_clothing_1: long (nullable = true)
 |

In [7]:
visits.count()

29875

#### Делаем фичи по времени

Из временных меток посещения сайтов сделайте следующие признаки (колонки):

число посещений по дням недели. То есть, постройте 7 признаков - "web_day_mon", "web_day_tue", ... , "web_day_sun" и прибавьте 1 для каждой метки, которая попадает на тот или иной день недели.
число посещений по часам в сутках. Аналогично дням недели: "web_hour_0", ... "web_hour_23".
долю посещений в рабочие часы "web_fraction_work_hours" (число посещений в рабочие часы/общее число посещений данного клиента) и долю посещений в вечерние часы "web_fraction_evening_hours", где рабочие часы - [9, 18), а вечерние - [18, 24)

In [56]:
spark.conf.set("spark.sql.session.timeZone", "UTC")

val log_time = log_3
        .select('uid,'timestamp)
        .withColumn("time_UTC", from_unixtime(col("timestamp")/1000,"yyyy-MM-dd HH:mm:ss"))

log_time = [uid: string, timestamp: bigint ... 1 more field]


[uid: string, timestamp: bigint ... 1 more field]

In [50]:
log_time.printSchema

root
 |-- uid: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- time_UTC: string (nullable = true)



In [57]:
log_time.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").show

+--------------------+-------------+-------------------+
|                 uid|    timestamp|           time_UTC|
+--------------------+-------------+-------------------+
|39d8aded-2111-42a...|1419590902274|2014-12-26 10:48:22|
|39d8aded-2111-42a...|1419590344372|2014-12-26 10:39:04|
|39d8aded-2111-42a...|1419590203997|2014-12-26 10:36:43|
|39d8aded-2111-42a...|1419589141914|2014-12-26 10:19:01|
|39d8aded-2111-42a...|1419588542058|2014-12-26 10:09:02|
|39d8aded-2111-42a...|1427181952000|2015-03-24 07:25:52|
|39d8aded-2111-42a...|1427181928000|2015-03-24 07:25:28|
|39d8aded-2111-42a...|1427181927000|2015-03-24 07:25:27|
+--------------------+-------------+-------------------+



##### Создаем udf-ки для дней, часов

In [65]:
val udf_day = udf((y: String) =>
    "web_day_".concat(
      y.toLowerCase())
    )

udf_day = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [105]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

val log_time2 = log_time.withColumn("input_timestamp",
    to_timestamp(col("time_UTC")))
    //.withColumn("week_day_number", date_format(col("input_timestamp"), "u"))
    .withColumn("week_day_abb", udf_day(date_format(col("input_timestamp"), "E")))
    //.withColumn("hour", hour(col("input_timestamp")))
    .withColumn("hour", hour(col("time_UTC")))
    .withColumn("web_hour", concat(lit("web_hour_"), col("hour").cast(StringType)))
    .withColumn("work_hours", expr("case when hour >=9 and hour < 18 then 1 else 0 end"))
    .withColumn("evening_hours", expr("case when hour >=18 and hour <= 23 then 1 else 0 end"))

log_time2 = [uid: string, timestamp: bigint ... 7 more fields]


[uid: string, timestamp: bigint ... 7 more fields]

In [110]:
log_time2.printSchema

root
 |-- uid: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- time_UTC: string (nullable = true)
 |-- input_timestamp: timestamp (nullable = true)
 |-- week_day_abb: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- web_hour: string (nullable = true)
 |-- work_hours: integer (nullable = false)
 |-- evening_hours: integer (nullable = false)



In [111]:
log_time2.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").show

+--------------------+-------------+-------------------+-------------------+------------+----+-----------+----------+-------------+
|                 uid|    timestamp|           time_UTC|    input_timestamp|week_day_abb|hour|   web_hour|work_hours|evening_hours|
+--------------------+-------------+-------------------+-------------------+------------+----+-----------+----------+-------------+
|39d8aded-2111-42a...|1419590902274|2014-12-26 10:48:22|2014-12-26 10:48:22| web_day_fri|  10|web_hour_10|         1|            0|
|39d8aded-2111-42a...|1419590344372|2014-12-26 10:39:04|2014-12-26 10:39:04| web_day_fri|  10|web_hour_10|         1|            0|
|39d8aded-2111-42a...|1419590203997|2014-12-26 10:36:43|2014-12-26 10:36:43| web_day_fri|  10|web_hour_10|         1|            0|
|39d8aded-2111-42a...|1419589141914|2014-12-26 10:19:01|2014-12-26 10:19:01| web_day_fri|  10|web_hour_10|         1|            0|
|39d8aded-2111-42a...|1419588542058|2014-12-26 10:09:02|2014-12-26 10:09:02|

In [112]:
// проверка
log_time2.groupBy("hour","work_hours", "evening_hours").count.sort('hour)show(25)

+----+----------+-------------+------+
|hour|work_hours|evening_hours| count|
+----+----------+-------------+------+
|   0|         0|            0| 60705|
|   1|         0|            0| 66886|
|   2|         0|            0| 69298|
|   3|         0|            0|105691|
|   4|         0|            0|146261|
|   5|         0|            0|187798|
|   6|         0|            0|240577|
|   7|         0|            0|273539|
|   8|         0|            0|296071|
|   9|         1|            0|270566|
|  10|         1|            0|291345|
|  11|         1|            0|292126|
|  12|         1|            0|279606|
|  13|         1|            0|306404|
|  14|         1|            0|281448|
|  15|         1|            0|290800|
|  16|         1|            0|287460|
|  17|         1|            0|289537|
|  18|         0|            1|317837|
|  19|         0|            1|311038|
|  20|         0|            1|252770|
|  21|         0|            1|178734|
|  22|         0|        

#### Собираем фичи по времени

In [128]:
val work_evening = log_time2
        .withColumn("1", lit(1))
        .groupBy('uid)
        .agg(
            sum("1").as("all_visit"),
            sum('work_hours).as("visit_work"),
            sum('evening_hours).as("visit_evening"))
        .withColumn("web_fraction_work_hours", col("visit_work")/col("all_visit"))
        .withColumn("web_fraction_evening_hours", col("visit_evening")/col("all_visit"))

work_evening = [uid: string, all_visit: bigint ... 4 more fields]


lastException: Throwable = null


[uid: string, all_visit: bigint ... 4 more fields]

In [129]:
work_evening.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").show


+--------------------+---------+----------+-------------+-----------------------+--------------------------+
|                 uid|all_visit|visit_work|visit_evening|web_fraction_work_hours|web_fraction_evening_hours|
+--------------------+---------+----------+-------------+-----------------------+--------------------------+
|39d8aded-2111-42a...|        8|         5|            0|                  0.625|                       0.0|
+--------------------+---------+----------+-------------+-----------------------+--------------------------+



In [134]:
val web_day = log_time2
            .withColumn("1", lit(1))
            .groupBy("uid")
            .pivot("week_day_abb")
            .sum("1")
            .na.fill(0)

web_day = [uid: string, web_day_fri: bigint ... 6 more fields]


[uid: string, web_day_fri: bigint ... 6 more fields]

In [138]:
web_day.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").show

+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|                 uid|web_day_fri|web_day_mon|web_day_sat|web_day_sun|web_day_thu|web_day_tue|web_day_wed|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|39d8aded-2111-42a...|          5|          0|          0|          0|          0|          3|          0|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+



In [136]:
val web_hour = log_time2
            .withColumn("1", lit(1))
            .groupBy("uid")
            .pivot("web_hour")
            .sum("1")
            .na.fill(0)

web_hour = [uid: string, web_hour_0: bigint ... 23 more fields]


[uid: string, web_hour_0: bigint ... 23 more fields]

In [139]:
web_hour.filter($"uid" ==="39d8aded-2111-42ab-9393-37c4f781d030").show

+--------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+----------+----------+
|                 uid|web_hour_0|web_hour_1|web_hour_10|web_hour_11|web_hour_12|web_hour_13|web_hour_14|web_hour_15|web_hour_16|web_hour_17|web_hour_18|web_hour_19|web_hour_2|web_hour_20|web_hour_21|web_hour_22|web_hour_23|web_hour_3|web_hour_4|web_hour_5|web_hour_6|web_hour_7|web_hour_8|web_hour_9|
+--------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+----------+----------+
|39d8aded-2111-42a...|         0|         0|          5|          0|          0|          0|     

In [140]:
println("web_day.count: ", web_day.count, "web_hour.count: ", web_hour.count)

(web_day.count: ,36138,web_hour.count: ,36138)


#### Соединяем всем фичи вместе

In [146]:
val all_feat = cl_topjoin2  //domain_features
        .join(work_evening.select('uid,'web_fraction_work_hours, 'web_fraction_evening_hours), Seq("uid"), "inner")
        .join(web_day, Seq("uid"), "inner")
        .join(web_hour, Seq("uid"), "inner") 
        .join(visits, Seq("uid"), "left") // из прошлой лабы
        .na.fill(0)

all_feat = [uid: string, domain_features: array<bigint> ... 673 more fields]


[uid: string, domain_features: array<bigint> ... 673 more fields]

In [147]:
all_feat.printSchema

root
 |-- uid: string (nullable = true)
 |-- domain_features: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- web_fraction_work_hours: double (nullable = false)
 |-- web_fraction_evening_hours: double (nullable = false)
 |-- web_day_fri: long (nullable = false)
 |-- web_day_mon: long (nullable = false)
 |-- web_day_sat: long (nullable = false)
 |-- web_day_sun: long (nullable = false)
 |-- web_day_thu: long (nullable = false)
 |-- web_day_tue: long (nullable = false)
 |-- web_day_wed: long (nullable = false)
 |-- web_hour_0: long (nullable = false)
 |-- web_hour_1: long (nullable = false)
 |-- web_hour_10: long (nullable = false)
 |-- web_hour_11: long (nullable = false)
 |-- web_hour_12: long (nullable = false)
 |-- web_hour_13: long (nullable = false)
 |-- web_hour_14: long (nullable = false)
 |-- web_hour_15: long (nullable = false)
 |-- web_hour_16: long (nullable = false)
 |-- web_hour_17: long (nullable = false)
 |-- web_hour_18: long (nullable = false)

In [39]:
all_feat.filter($"buy_cameras_0".isNull).count()
// нуллы заполнил

0

#### Пишем
Сохраните в формате parquet по пути /user/name.surname/features

In [149]:
all_feat
    .write
    .mode("overwrite")
    .parquet("/user/vladimir.dyukarev/features")

In [148]:
println("hdfs dfs -ls /user/vladimir.dyukarev/features".!!)
//println("hdfs dfs -rm -r -skipTrash /user/vladimir.dyukarev/features/*".!!)

Deleted /user/vladimir.dyukarev/features/_SUCCESS
Deleted /user/vladimir.dyukarev/features/part-00000-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00001-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00002-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00003-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00004-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00005-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00006-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00007-56999d37-5d1d-47d6-a81f-652a95febc0a-c000.snappy.parquet
Deleted /user/vladimir.dyukarev/features/part-00008-56999d37-5d1d-47d6-a81f-65